# Working With Dates in Pandas

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Datetime Types

In [2]:
pd.to_datetime('Jan 1 1970')

Timestamp('1970-01-01 00:00:00')

In [3]:
# pd.to_datetime('Jan:1:1970')

In [4]:
pd.to_datetime('Jan:1:1970', format='%b:%d:%Y')

Timestamp('1970-01-01 00:00:00')

In [5]:
url = "https://gist.githubusercontent.com/ryanorsinger/b309f8db19e0ca71b213d4877d835e77/raw/f5841017310e2f4ca070b313529ceec2375336ba/coffee_consumption.csv"
df = pd.read_csv(url)
df.head()

,date,coffee_consumption
0,2019-01-01,14.301915
1,2019-01-02,12.905900
2,2019-01-03,10.046015
3,2019-01-04,6.354805
4,2019-01-07,8.545563


In [6]:
df.date = pd.to_datetime(df.date)
df.dtypes

date                  datetime64[ns]
coffee_consumption           float64
dtype: object

In [11]:
pd.__version__

'1.2.4'

## `.dt` accessor

In [7]:
df.date.dt.day_name()

0        Tuesday
1      Wednesday
2       Thursday
3         Friday
4         Monday
         ...    
256    Wednesday
257     Thursday
258       Friday
259       Monday
260      Tuesday
Name: date, Length: 261, dtype: object

How many observations are there for each month? Each weekday?

## DateTime Indexes

In [8]:
df = df.set_index('date').sort_index()
df

,coffee_consumption
date,
2019-01-01,14.301915
2019-01-02,12.905900
2019-01-03,10.046015
2019-01-04,6.354805
2019-01-07,8.545563
...,...
2019-12-25,12.250875
2019-12-26,7.513206
2019-12-27,9.464345


In [12]:
# earliest + latest dates
df.index.min(), df.index.max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [15]:
# Month of March
df.loc['2019-03']

,coffee_consumption
date,
2019-03-01,9.109061
2019-03-04,9.054681
2019-03-05,9.915438
2019-03-06,11.254156
2019-03-07,5.680393
2019-03-08,8.823756
2019-03-11,11.296137
2019-03-12,6.552509
2019-03-13,10.784550


In [16]:
# Subset of march
# One date to another
df.loc['2019-03-01':"2019-03-13"]

,coffee_consumption
date,
2019-03-01,9.109061
2019-03-04,9.054681
2019-03-05,9.915438
2019-03-06,11.254156
2019-03-07,5.680393
2019-03-08,8.823756
2019-03-11,11.296137
2019-03-12,6.552509
2019-03-13,10.784550


### Resampling

Upsampling: generally increases the number of rows and introduces nulls

In [17]:
by_day = df.asfreq('D')
by_day

,coffee_consumption
date,
2019-01-01,14.301915
2019-01-02,12.905900
2019-01-03,10.046015
2019-01-04,6.354805
2019-01-05,NaN
...,...
2019-12-27,9.464345
2019-12-28,NaN
2019-12-29,NaN


In [18]:
# Forward fill vs back fill
by_day.assign(
    ffill=lambda df: df.coffee_consumption.ffill(),
    bfill=lambda df: df.coffee_consumption.bfill()
).head(15)

,coffee_consumption,ffill,bfill
date,,,
2019-01-01,14.301915,14.301915,14.301915
2019-01-02,12.905900,12.905900,12.905900
2019-01-03,10.046015,10.046015,10.046015
2019-01-04,6.354805,6.354805,6.354805
2019-01-05,NaN,6.354805,8.545563
2019-01-06,NaN,6.354805,8.545563
2019-01-07,8.545563,8.545563,8.545563
2019-01-08,9.410101,9.410101,9.410101
2019-01-09,7.219271,7.219271,7.219271


In [19]:
df = df.fillna(0)

Downsampling: reduce the number of rows and aggregate 
Like a group by

In [20]:
# average of 3 weeks
df.resample('3W').mean()

,coffee_consumption
date,
2019-01-06,10.902159
2019-01-27,9.744190
2019-02-17,10.855301
2019-03-10,9.413472
2019-03-31,11.062768
2019-04-21,10.222518
2019-05-12,10.092044
2019-06-02,11.052819
2019-06-23,9.592148


In [21]:
df.resample('M').sum()

,coffee_consumption
date,
2019-01-31,233.926749
2019-02-28,206.038853
2019-03-31,219.779000
2019-04-30,222.030626
2019-05-31,248.480093
2019-06-30,183.656493
2019-07-31,248.267463
2019-08-31,224.007066
2019-09-30,215.689004


### Rolling Windows

In [25]:
# 3 day moving average
df.rolling(3).mean()

,coffee_consumption
date,
2019-01-01,NaN
2019-01-02,NaN
2019-01-03,12.417943
2019-01-04,9.768907
2019-01-07,8.315461
...,...
2019-12-25,10.595246
2019-12-26,10.045494
2019-12-27,9.742809


In [26]:
# Moving monthly sum
df.rolling(30).sum()

,coffee_consumption
date,
2019-01-01,NaN
2019-01-02,NaN
2019-01-03,NaN
2019-01-04,NaN
2019-01-07,NaN
...,...
2019-12-25,312.361955
2019-12-26,308.561795
2019-12-27,311.635037


### Lagging and Lead

In [27]:
# diff => difference between previous day and current day
# shift => push previous day forward
pd.concat([
    df.coffee_consumption,
    df.coffee_consumption.diff(),
    df.coffee_consumption.shift(),
], axis=1)

,coffee_consumption,coffee_consumption,coffee_consumption
date,,,
2019-01-01,14.301915,NaN,NaN
2019-01-02,12.905900,-1.396015,14.301915
2019-01-03,10.046015,-2.859885,12.905900
2019-01-04,6.354805,-3.691210,10.046015
2019-01-07,8.545563,2.190757,6.354805
...,...,...,...
2019-12-25,12.250875,1.878475,10.372400
2019-12-26,7.513206,-4.737669,12.250875
2019-12-27,9.464345,1.951140,7.513206


In [ ]:
# different time periods

In [28]:
df

,coffee_consumption
date,
2019-01-01,14.301915
2019-01-02,12.905900
2019-01-03,10.046015
2019-01-04,6.354805
2019-01-07,8.545563
...,...
2019-12-25,12.250875
2019-12-26,7.513206
2019-12-27,9.464345


## Strftime

In [29]:
df.index.strftime('%B %d, %Y')[:4]

Index(['January 01, 2019', 'January 02, 2019', 'January 03, 2019',
       'January 04, 2019'],
      dtype='object', name='date')

## Timezones

In [ ]:
np.random.seed(123)

idx = pd.date_range('now', freq='H', periods=100)
hourly_df = pd.DataFrame({'x': np.random.choice(list('abc'), 100), 'y': np.random.rand(100)}, index=idx)

In [ ]:
hourly_df.index.tz is None

In [ ]:
hourly_df.tz_localize('America/Chicago')

In [ ]:
hourly_df.tz_localize(None)

In [ ]:
hourly_df = hourly_df.tz_localize('America/New_York')
hourly_df

In [ ]:
hourly_df.tz_convert('America/Los_Angeles')

[Wikipedia: List of Time Zones](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)

## Timedeltas

In [ ]:
start_date = pd.to_datetime('20210614')
today = pd.to_datetime('20210930')
graduation_date = pd.to_datetime('20211207')

In [ ]:
df['days_since_max'] = (df.index.max() - df.index) // pd.Timedelta('1d')
df.tail(10)

## Plotting

In [ ]:
df.plot()

In [ ]:
df.resample('3M').mean().plot(marker='o')